In [1]:
# import requests

# url = "https://raw.githubusercontent.com/ltdaovn/dataset/master/NLP%20with%20Disaster%20Tweets/Disaster_Tweets_train.csv"
# response = requests.get(url)

In [2]:
# with open('Disaster_Tweets_train.csv', 'wb') as file:
#     file.write(response.content)

In [3]:
import pandas as pd

data = pd.read_csv("Disaster_Tweets_train.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [7]:
import re
import string
import json

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

with open('abbreviations.json', 'r') as f:
    contractions = json.load(f)


def expand_contractions(text):
    for contraction, full_form in contractions.items():
        text = re.sub(r'\b' + re.escape(contraction) + r'\b', full_form, text)

    return text


def split_hashtag(hashtag):
    hashtag = hashtag[1:]
    split_words = re.sub(r'([a-z])([A-Z])', r'\1 \2', hashtag)
    split_words = re.sub(r'[^a-zA-Z]', ' ', split_words)
    return split_words

def preprocess_text(text):
    text = text.lower()
    
    text = expand_contractions(text)
    
    hashtags = re.findall(r'#\w+', text)
    for hashtag in hashtags:
        split_words = split_hashtag(hashtag)
        text = text.replace(hashtag, split_words)
    
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http[s]?://\S+', '', text)
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    text = re.sub(r'\#\w+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token)
              for token in tokens if token not in stop_words]
    return ' '.join(tokens)

In [8]:
preprocess_text(
    'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all')

'deed reason earthquake may allah forgive u'

In [10]:
data['cleaned_text'] = data['text'].apply(preprocess_text)

output_file = 'cleaned_data.csv'
data[['id', 'keyword', 'location', 'cleaned_text', 'target']].to_csv(
    output_file, index=False)

print("Data has been cleaned and saved to", output_file)

Data has been cleaned and saved to cleaned_data.csv
